### Use !pip install to download and install necessary packages.

In [179]:
# EXAMPLE -> !pip install keras
# Necessary Packages include -> pandas, plotly, numpy, seaborn, math, keras, sklearn, matplotlib
import numpy as np
import pandas as pd
import plotly.offline as py 
import plotly.graph_objs as go
import seaborn as sns
import math
import keras
import tensorflow

from numpy.random import seed
from tensorflow import set_random_seed
seed = np.random.seed(432)
set_random_seed(seed)

from pandas import DataFrame
from pandas import read_csv
from numpy import concatenate 
from numpy import newaxis
from math import sqrt 
from matplotlib import pyplot 
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

from keras.models import Sequential 
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Flatten
from keras.utils import plot_model

py.init_notebook_mode(connected = True)
%matplotlib inline

#### Read in Data, Check NAs, Replace

In [181]:
df = pd.read_csv('FAR_test2.txt', sep = '|')
df['REVENUE'].replace(0, np.nan, inplace = True)
df['REVENUE'].fillna(method = 'ffill', inplace = True)

uniquecust = list(df.CUSTOMER_CODE.unique())
#Turn NAs to 0 
df.isnull().values.sum()
df = df.replace(np.nan, 0, regex=True)

#### Parameters

In [257]:
# Initial Parameters #
train_size = 157
layers = [100,55,50,1]
drop_out = [0.2,0.3]
epochs = 10
batch_size = 1
look_back = 4

fore = pd.array.empty
predicted = []

In [262]:
scaleddf(df)

469
Epoch 1 / 10
Epoch 1/1
153/153 [==============================] - 2s 16ms/step - loss: 0.0744
Epoch 2 / 10
Epoch 1/1
153/153 [==============================] - 2s 11ms/step - loss: 0.0611
Epoch 3 / 10
Epoch 1/1
153/153 [==============================] - 2s 13ms/step - loss: 0.0720
Epoch 4 / 10
Epoch 1/1
153/153 [==============================] - 2s 12ms/step - loss: 0.0716
Epoch 5 / 10
Epoch 1/1
153/153 [==============================] - 2s 12ms/step - loss: 0.0587
Epoch 6 / 10
Epoch 1/1
153/153 [==============================] - 2s 11ms/step - loss: 0.0497
Epoch 7 / 10
Epoch 1/1
153/153 [==============================] - 2s 12ms/step - loss: 0.0600
Epoch 8 / 10
Epoch 1/1
153/153 [==============================] - 2s 12ms/step - loss: 0.0559
Epoch 9 / 10
Epoch 1/1
153/153 [==============================] - 2s 11ms/step - loss: 0.0612
Epoch 10 / 10
Epoch 1/1
153/153 [==============================] - 2s 12ms/step - loss: 0.0662
Train RMSE: 0.251


AttributeError: 'property' object has no attribute 'append'

In [261]:
def scaleddf(dataframe):
    for custcode in uniquecust:
        # Normalize
        values = dataframe['REVENUE'].values.reshape(-1,1)
        values = values.astype('float32')
        scaler = MinMaxScaler(feature_range=(0,1))
        scaled = scaler.fit_transform(values)
        
        # Create Tensor
        train_size = int(len(scaled) * 1)
        train = scaled[0:train_size, :]
        trainX, trainY = create_dataset(train, look_back)
        trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
        #print(trainX.shape, trainY.shape)
        
        # Create/Run model
        model_stateful = create_model(stateful = True)
        runmodel(epochs, batch_size)
        
        #Predict on Train Set 
        trainPredict = model_stateful.predict(trainX, batch_size = batch_size)
        rmse_train = sqrt(mean_squared_error(trainPredict, trainY))
        print('Train RMSE: %.3f' % rmse_train)

        trainPredict2 = scaler.inverse_transform(trainPredict)
        trainY2 = scaler.inverse_transform([trainY])
        trainYInverse = np.reshape(trainY2, (len(trainPredict), 1))
        
        #Produce Forecasts
        forecastdf(trainX)
        
        #Append Forecasts - 1 Array
        fore.append(predicted)
        model.reset_states()
    return 

## FUNCTIONS


In [192]:
def create_dataset(dataset, look_back):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i+look_back, 0])
    print(len(dataY))
    return np.array(dataX), np.array(dataY)

In [199]:
def create_model(stateful):
    model = Sequential()
#Cell 1
    model.add(LSTM(
        units = layers[0],
        return_sequences = True,
        batch_input_shape = (1, trainX.shape[1], trainX.shape[2]),
        stateful = stateful 
    ))
    model.add(Dropout(drop_out[0]))
#Cell 2
    model.add(LSTM(
        units=layers[1], 
        return_sequences = True,
        stateful = stateful
    ))        
    model.add(Dropout(drop_out[1]))
#Cell 3
    model.add(LSTM(
        units=layers[2],
        return_sequences = True,    
        stateful = stateful
    ))
#Output Layer
    model.add(Flatten())
    model.add(Dense(
        units = layers[3],
        activation = 'linear'
    ))
    model.compile(loss = 'mae', optimizer = 'adam')   
    return model

print('Creating Stateful Model..')

Creating Stateful Model..


In [210]:
def runmodel(epochs, batch_size):
    for i in range(epochs):
        print('Epoch', i + 1, '/', epochs)
        model_stateful.fit(trainX,
              trainY,
              batch_size=batch_size,
              epochs=1,
              verbose=1,
              #validation_data=(testX, testY),
              shuffle=False)
        model_stateful.reset_states()
    return 

In [260]:
def forecastdf(df):
    curr_frame = trainX[0]
    for i in range(len(df)):                                                     
        predicted.append(model_stateful.predict(curr_frame[newaxis,:,:], batch_size = batch_size)[0,0]) 
        curr_frame = curr_frame[1:]                                                  
        curr_frame = np.insert(curr_frame, len(df[0])-1, predicted[-1], axis = 0)
        forecastPredict = scaler.inverse_transform([predicted])
        forecasts = np.reshape(forecastPredict, (len(predicted), 1))
        forecasts2 = forecasts[0:52] 
    return